# Funnel Queries (no funnel models)

In [ ]:
import os
from pathlib import Path

from dotenv import find_dotenv, load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

Develop **product funnel** queries without using funnel-specific models.

### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.
2. After joining events and addresses, the hierarchy is state > user > product > session > event.
3. Expected Outputs
   - Funnel (from project walkthrough video)
     - packages shipped = 335
     - checkouts = 361
     - add to carts = 986<sup>[1](#myfootnote1)</sup>
     - page views = 1.87k<sup>[1](#myfootnote1)</sup>
   -  checkouts by state
      - Texas = 87
      - California = 56
      - Florida = 21
      - Virginia = 19
      - Arizona = 17
      - Illinois = 16
      - New York = 15
      - District of Columbia = 13
      - Pennsylvania = 10
      - Massachusetts = 9
      - South Carolina = 9
   - grand totals
     - sessions = 578
     - checkout sessions = 361

   <a name="myfootnote1">1</a>: this is the number of distinct events, not the number of distinct sessions; in the assignment, we are required to get the number of sessions

## User Inputs

In [ ]:
min_date = '2021-02-09'
max_date = '2021-02-12'

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA"),
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Connect to DuckDB database

In [ ]:
%sql engine --alias connection

## Exploratory Data Analysis

From the week 4 notes

> At a high level, a session is a group of actions taken by a specific user in a given time frame. A common definition of a session is from the time someone comes to the website/app to when they have their last activity + 30 minutes. In other words, from the first activity to 30 minutes after the last activity.

Show all events from sessions ending in a purchase

In [ ]:
%%sql
SELECT *
FROM int_product_purchases_filtered
WHERE session_id IN (
    'a646b9aa-0044-4fbd-8bae-a1018f5d4ace',
    'bb40cd22-b3aa-431e-af50-d2726bfc45d0'
)
ORDER BY session_id, created_at

Show the number of sessions with more than one user

In [ ]:
%%sql
SELECT COUNT(DISTINCT(user_id)) AS num_users
FROM stg_postgres_events
GROUP BY session_id
HAVING num_users > 1

Get the number of (a) sessions and (b) users with a page view or add-to-carts per product

In [ ]:
%%sql
WITH user_addresses AS (
    SELECT u.user_id,
           a.state AS state_name
    FROM (
        SELECT DISTINCT(user_id) AS user_id
        FROM stg_postgres_events
    ) AS u
    INNER JOIN stg_postgres_users us USING (user_id)
    INNER JOIN stg_postgres_addresses a USING (address_id)
),
t0 AS (
    SELECT ua.state_name,
           p.name AS product_name,
           event_type,
           COUNT(DISTINCT(session_id)) As num_sessions,
           COUNT(DISTINCT(user_id)) As num_users
    FROM stg_postgres_events e
    INNER JOIN stg_postgres_products p USING (product_id)
    INNER JOIN user_addresses ua USING(user_id)
    WHERE product_name IN ('Aloe Vera', 'Orchid', 'Bamboo')
    GROUP BY ALL
    ORDER BY product_name, num_sessions DESC
),
t1 AS (
    SELECT product_name,
           event_type,
           SUM(num_sessions) AS num_sessions,
           SUM(num_users) AS num_users
    FROM t0
    GROUP BY ALL
    ORDER BY product_name, num_sessions DESC
)
SELECT *
FROM t1

Get event summary

In [ ]:
%%sql
SELECT COUNT(DISTINCT(session_id)) AS num_sessions,
       COUNT(DISTINCT(user_id)) AS num_users,
       COUNT(DISTINCT(event_id)) AS num_events
FROM stg_postgres_events

Get temporal attribues for all sessions

In [ ]:
%%sql
WITH session_temporal_attributes AS (
    SELECT session_id,
           datediff(
               'second', MIN(created_at), MAX(created_at)
           ) AS session_length,
           MIN(created_at) AS started_at,
           MAX(created_at) AS ended_at
    FROM stg_postgres_events
    GROUP BY ALL
)
SELECT *
FROM session_temporal_attributes

### All Events

Set the maximum number of rows to be displayed to `None` (shows all rows)

In [ ]:
%config SqlMagic.displaylimit = None

Get product ID for single product

In [ ]:
%%sql
SELECT *
FROM stg_postgres_products
WHERE name = 'Pothos'

Show all events for single state-product combination

In [ ]:
%%sql
WITH user_addresses AS (
    SELECT u.user_id,
           a.state AS state_name
    FROM (
        SELECT DISTINCT(user_id) AS user_id
        FROM stg_postgres_events
    ) AS u
    INNER JOIN stg_postgres_users us USING (user_id)
    INNER JOIN stg_postgres_addresses a USING (address_id)
),
texas_pothos_sessions AS (
    SELECT DISTINCT(session_id) AS session_id
    FROM int_product_purchases_filtered e
    INNER JOIN user_addresses ua USING (user_id)
    WHERE state_name = 'Texas'
    AND product_id = '4cda01b9-62e2-46c5-830f-b7f262a58fb1'
)
SELECT * EXCLUDE(page_url, event_id, order_id, user_id, product_id),
       name AS product_name
FROM stg_postgres_events
INNER JOIN texas_pothos_sessions USING (session_id)
INNER JOIN user_addresses ua USING (user_id)
LEFT JOIN stg_postgres_products p USING (product_id)
ORDER BY session_id, created_at

Show products that were added to a cart or checked out in Indiana

In [ ]:
%%sql
WITH user_addresses AS (
    SELECT u.user_id,
           a.state AS state_name
    FROM (
        SELECT DISTINCT(user_id) AS user_id
        FROM stg_postgres_events
    ) AS u
    INNER JOIN stg_postgres_users us USING (user_id)
    INNER JOIN stg_postgres_addresses a USING (address_id)
),
t1 AS (
    SELECT e.* EXCLUDE(user_id, page_url, order_id, event_id),
           p.name AS product_name,
           ua.state_name
    FROM stg_postgres_events e
    INNER JOIN user_addresses ua USING (user_id)
    LEFT JOIN stg_postgres_products p USING (product_id)
    WHERE state_name = 'Indiana'
)
SELECT *
FROM t1
WHERE event_type IN ('page_view', 'add_to_cart', 'checkout')

**Observations**

1. There were six sessions with events.
2. There were six orders placed, which each session ended in an order.
3. Over all six orders, there were 10 products that were ordered (10 occurrences of `add_to_cart` before an occurrence of `checkout`).

Count number of events from single-day sessions

In [ ]:
%%sql
WITH events AS (
    SELECT * EXCLUDE(page_url, order_id, event_id)
    FROM stg_postgres_events
),
multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
),
single_day_session_events AS (
    SELECT *
    FROM events
    LEFT JOIN multi_day_sessions USING (session_id)
    WHERE event_type != 'package_shipped'
    AND is_multi_day_session IS NULL
)
SELECT COUNT(*)
FROM single_day_session_events

### Funnels

Set the maximum number of rows to be displayed (default is 10 rows)

In [ ]:
%config SqlMagic.displaylimit = 10

Get overall funnel

In [ ]:
%%sql
WITH multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM stg_postgres_events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
)
SELECT event_type,
       COUNT(DISTINCT(session_id)) AS num_sessions
FROM stg_postgres_events e
LEFT JOIN multi_day_sessions USING (session_id)
WHERE event_type NOT IN ('package_shipped')
AND is_multi_day_session IS NULL
GROUP BY ALL
ORDER BY num_sessions DESC

Get overall funnel, including (a) all four `event_type`s and (b) multi-day sessions (to compare to expected from **About** section)

In [ ]:
%%sql
WITH multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM stg_postgres_events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
)
SELECT event_type,
       COUNT(DISTINCT(session_id)) AS num_sessions
FROM stg_postgres_events e
LEFT JOIN multi_day_sessions USING (session_id)
GROUP BY ALL
ORDER BY num_sessions DESC

Get overall daily funnel

In [ ]:
%%sql
WITH multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM stg_postgres_events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
)
SELECT TO_DATE(created_at) AS created_at_date,
       event_type,
       COUNT(DISTINCT(session_id)) AS num_sessions
FROM stg_postgres_events e
LEFT JOIN multi_day_sessions USING (session_id)
WHERE event_type NOT IN ('package_shipped')
AND is_multi_day_session IS NULL
GROUP BY ALL
ORDER BY created_at_date, num_sessions DESC

Get state funnel

In [ ]:
%%sql
WITH user_addresses AS (
    SELECT u.user_id,
           a.state AS state_name
    FROM (
        SELECT DISTINCT(user_id) AS user_id
        FROM stg_postgres_events
    ) AS u
    INNER JOIN stg_postgres_users us USING (user_id)
    INNER JOIN stg_postgres_addresses a USING (address_id)
),
multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM stg_postgres_events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
)
SELECT ua.state_name,
       event_type,
       COUNT(DISTINCT(session_id)) AS num_sessions
FROM stg_postgres_events e
LEFT JOIN multi_day_sessions USING (session_id)
INNER JOIN user_addresses ua USING (user_id)
WHERE event_type != 'package_shipped'
AND is_multi_day_session IS NULL
GROUP BY ALL
ORDER BY state_name, num_sessions DESC

Get state funnel for checkouts including multi-day sessions (to compare to expected from **About** section)

In [ ]:
%%sql
WITH user_addresses AS (
    SELECT u.user_id,
           a.state AS state_name
    FROM (
        SELECT DISTINCT(user_id) AS user_id
        FROM stg_postgres_events
    ) AS u
    INNER JOIN stg_postgres_users us USING (user_id)
    INNER JOIN stg_postgres_addresses a USING (address_id)
),
multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM stg_postgres_events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
)
SELECT ua.state_name,
       event_type,
       COUNT(DISTINCT(session_id)) AS num_sessions
FROM stg_postgres_events e
LEFT JOIN multi_day_sessions USING (session_id)
INNER JOIN user_addresses ua USING (user_id)
WHERE event_type = 'checkout'
-- AND is_multi_day_session IS NULL
GROUP BY ALL
ORDER BY num_sessions DESC

Get state daily funnel

In [ ]:
%%sql
WITH user_addresses AS (
    SELECT u.user_id,
           a.state AS state_name
    FROM (
        SELECT DISTINCT(user_id) AS user_id
        FROM stg_postgres_events
    ) AS u
    INNER JOIN stg_postgres_users us USING (user_id)
    INNER JOIN stg_postgres_addresses a USING (address_id)
),
event_order_mapper AS (
    SELECT *
    FROM (
        -- VALUES ('page_view', '01: page_view'),
        -- ('add_to_cart', '02: add_to_cart'),
        -- ('checkout', '03: checkout')
        VALUES ('page_view', 1, 'page_view'),
        ('add_to_cart', 2, 'add_to_cart'),
        ('checkout', 3, 'checkout')
    ) AS t0 (event_type, step_order, event_type)
),
multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM stg_postgres_events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
)
SELECT ua.state_name,
       TO_DATE(created_at) AS created_at_date,
       step_order,
       event_type,
       COUNT(DISTINCT(session_id)) AS num_sessions
FROM stg_postgres_events e
LEFT JOIN multi_day_sessions USING (session_id)
INNER JOIN user_addresses ua USING (user_id)
INNER JOIN event_order_mapper em USING (event_type)
WHERE event_type != 'package_shipped'
AND is_multi_day_session IS NULL
GROUP BY ALL
ORDER BY state_name, created_at_date, step_order

### Funnels Without Checkouts

Get state-product funnel

In [ ]:
%%sql
WITH multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM stg_postgres_events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
),
user_addresses AS (
    SELECT u.user_id,
           a.state AS state_name
    FROM (
        SELECT DISTINCT(user_id) AS user_id
        FROM stg_postgres_events
        LEFT JOIN multi_day_sessions USING (session_id)
        WHERE event_type != 'package_shipped'
        AND is_multi_day_session IS NULL
    ) AS u
    INNER JOIN stg_postgres_users us USING (user_id)
    INNER JOIN stg_postgres_addresses a USING (address_id)
)
SELECT ua.state_name,
       p.name AS product_name,
       event_type,
       COUNT(DISTINCT(session_id)) AS num_sessions
FROM stg_postgres_events e
LEFT JOIN multi_day_sessions USING (session_id)
INNER JOIN user_addresses ua USING (user_id)
INNER JOIN stg_postgres_products p USING (product_id)
WHERE is_multi_day_session IS NULL
GROUP BY ALL
ORDER BY state_name, product_name, num_sessions DESC

Get product funnel

In [ ]:
%%sql
WITH multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM stg_postgres_events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
)
SELECT p.name AS product_name,
       event_type,
       COUNT(DISTINCT(session_id)) AS num_sessions
FROM stg_postgres_events e
LEFT JOIN multi_day_sessions USING (session_id)
INNER JOIN stg_postgres_products p USING (product_id)
WHERE is_multi_day_session IS NULL
GROUP BY ALL
ORDER BY product_name, num_sessions DESC

Get product daily funnel

In [ ]:
%%sql
WITH multi_day_sessions AS (
    SELECT session_id,
           True AS is_multi_day_session
    FROM stg_postgres_events
    WHERE event_type != 'package_shipped'
    GROUP BY ALL
    HAVING COUNT(DISTINCT(TO_DATE(created_at))) > 1
)
SELECT p.name AS product_name,
       TO_DATE(created_at) AS created_at_date,
       event_type,
       COUNT(DISTINCT(session_id)) AS num_sessions
FROM stg_postgres_events e
LEFT JOIN multi_day_sessions USING (session_id)
INNER JOIN stg_postgres_products p USING (product_id)
WHERE is_multi_day_session IS NULL
GROUP BY ALL
ORDER BY product_name, created_at_date, num_sessions DESC

## Queries

Set the maximum number of rows to be displayed to `None` (shows all rows)

In [ ]:
%config SqlMagic.displaylimit = None

In [ ]:
%%sql
/* ============= INTERMEDIATE DATA MODEL STARTS HERE ============= */
WITH events AS (
    SELECT session_id,
           user_id,
           -- event_id,
           TO_DATE(created_at) AS created_at_date,
           created_at,
           event_type,
           product_id,
           is_purchased
    FROM int_product_purchases_filtered
),
non_events AS (
    SELECT session_id,
           user_id,
           -- event_id,
           TO_DATE(created_at) AS created_at_date,
           created_at,
           event_type,
           product_id
    FROM int_product_non_purchases_filtered
),
non_purchase_sessions_page_view_counts AS (
    SELECT session_id,
           user_id,
           -- event_id,
           product_id,
           created_at_date,
           COUNT(*) AS num_page_views
    FROM non_events
    WHERE event_type = 'page_view'
    GROUP BY ALL
),
non_purchase_sessions_add_to_cart_counts AS (
    SELECT session_id,
           user_id,
           -- event_id,
           product_id,
           created_at_date,
           COUNT(*) AS num_add_to_carts
    FROM non_events
    WHERE event_type = 'add_to_cart'
    GROUP BY ALL
),
purchase_sessions_cart_checkout_counts AS (
    SELECT session_id,
           user_id,
           -- event_id,
           product_id,
           created_at_date,
           1 AS num_add_to_carts,
           MAX(is_purchased) AS num_checkouts
    FROM events
    WHERE event_type = 'add_to_cart'
    AND product_id IS NOT NULL
    GROUP BY ALL
),
purchase_sessions_page_view_counts AS (
    SELECT session_id,
           user_id,
           -- event_id,
           product_id,
           created_at_date,
           COUNT(*) AS num_page_views
    FROM events
    WHERE event_type = 'page_view'
    GROUP BY ALL
),
daily_user_sessions AS (
    SELECT session_id,
           user_id,
           product_id,
           created_at_date,
           num_page_views,
           num_add_to_carts,
           num_checkouts
    FROM purchase_sessions_page_view_counts
    LEFT JOIN purchase_sessions_cart_checkout_counts
        USING (session_id, user_id, product_id, created_at_date)
    UNION ALL
    SELECT session_id,
           user_id,
           product_id,
           created_at_date,
           num_page_views,
           num_add_to_carts,
           0 AS num_checkouts
    FROM non_purchase_sessions_page_view_counts
    LEFT JOIN non_purchase_sessions_add_to_cart_counts
        USING (session_id, user_id, product_id, created_at_date)
),
daily_user_sessions_no_nulls AS (
    SELECT session_id,
           user_id,
           product_id,
           created_at_date,
           ZEROIFNULL(num_page_views) AS num_page_views,
           ZEROIFNULL(num_add_to_carts) AS num_add_to_carts,
           ZEROIFNULL(num_checkouts) AS num_checkouts
    FROM daily_user_sessions
),
/* ============= INTERMEDIATE DATA MODEL ENDS HERE ============= */
/* ============= MARTS fct_sessions_daily DATA MODEL STARTS HERE ============= */
users AS (
    SELECT user_id,
           address_id
    FROM stg_postgres_users
),
addresses AS (
    SELECT address_id,
           state AS state_name
    FROM stg_postgres_addresses
),
products AS (
    SELECT product_id,
           name AS product_name
    FROM stg_postgres_products
),
event_users AS (
    SELECT DISTINCT(user_id) AS user_id
    FROM stg_postgres_events
),
user_addresses AS (
     SELECT u.user_id,
            a.state_name
     FROM event_users AS u
     INNER JOIN users us USING (user_id)
     INNER JOIN addresses a USING (address_id)
),
daily_user_sessions_named AS (
    SELECT s.session_id,
           s.user_id,
           -- event_id,
           ua.state_name,
           p.product_name,
           s.created_at_date,
           num_page_views,
           num_add_to_carts,
           num_checkouts
    FROM daily_user_sessions_no_nulls AS s
    INNER JOIN user_addresses ua USING (user_id)
    INNER JOIN products p USING (product_id)
),
/* ============= MARTS fct_user_sessions DATA MODEL ENDS HERE ============= */
/* ============= BI TOOL STARTS HERE ============= */
daily_user_sessions_filtered AS (
    SELECT *
    FROM daily_user_sessions_named
    WHERE created_at_date BETWEEN '{{ min_date }}' AND '{{ max_date }}'
),
overall AS (
    SELECT ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT COUNT(DISTINCT(session_id)) AS page_views,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
    ) t1
    LEFT JOIN (
        SELECT COUNT(DISTINCT(session_id)) AS add_to_carts,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
    ) t2 USING (row_num)
    LEFT JOIN (
        SELECT COUNT(DISTINCT(session_id)) AS checkouts,
               1 AS row_num
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
    ) t3 USING (row_num)
    ORDER BY checkouts DESC
),
by_state AS (
    SELECT state_name,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (state_name)
    LEFT JOIN (
        SELECT state_name,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (state_name)
    ORDER BY checkouts DESC
),
by_product AS (
    SELECT product_name,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (product_name)
    LEFT JOIN (
        SELECT product_name,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (product_name)
    ORDER BY checkouts DESC
),
overall_daily AS (
    SELECT created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (created_at_date)
    LEFT JOIN (
        SELECT created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (created_at_date)
    ORDER BY created_at_date, checkouts DESC
),
by_state_daily AS (
    SELECT state_name,
           created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (state_name, created_at_date)
    LEFT JOIN (
        SELECT state_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (state_name, created_at_date)
    ORDER BY state_name, created_at_date, checkouts DESC
),
by_product_daily AS (
    SELECT product_name,
           created_at_date,
           ZEROIFNULL(page_views) AS page_views,
           ZEROIFNULL(add_to_carts) AS add_to_carts,
           ZEROIFNULL(checkouts) AS checkouts
    FROM (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS page_views
        FROM daily_user_sessions_filtered
        WHERE num_page_views > 0
        GROUP BY ALL
    ) t1
    LEFT JOIN (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS add_to_carts
        FROM daily_user_sessions_filtered
        WHERE num_add_to_carts > 0
        GROUP BY ALL
    ) t2 USING (product_name, created_at_date)
    LEFT JOIN (
        SELECT product_name,
               created_at_date,
               COUNT(DISTINCT(session_id)) AS checkouts
        FROM daily_user_sessions_filtered
        WHERE num_checkouts > 0
        GROUP BY ALL
    ) t3 USING (product_name, created_at_date)
    ORDER BY created_at_date, checkouts DESC
),
overall_pivotted AS (
    SELECT LOWER(event_type) AS event_type,
           number
    FROM overall
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_state_pivotted AS (
    SELECT LOWER(event_type) AS event_type,
           number
    FROM by_state
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_product_pivotted AS (
    SELECT LOWER(event_type) AS event_type,
           number
    FROM by_product
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
overall_daily_pivotted AS (
    SELECT created_at_date,
           LOWER(event_type) AS event_type,
           number
    FROM overall_daily
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_state_daily_pivotted AS (
    SELECT state_name,
           created_at_date,
           LOWER(event_type) AS event_type,
           number
    FROM by_state_daily
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
),
by_product_daily_pivotted AS (
    SELECT product_name,
           created_at_date,
           LOWER(event_type) AS event_type,
           number
    FROM by_product_daily
    UNPIVOT (number FOR event_type IN (page_views, add_to_carts, checkouts))
)
/* ============= BI TOOL ENDS HERE ============= */
SELECT *
FROM overall_daily

## Disconnect

Close connection

In [ ]:
%sql --close connection